In [0]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import lit, col, concat
from functools import reduce

In [0]:
class Refine:
    def __init__(self, df: DataFrame):
        self.df = df

    def renamecols(self, colmap):
        self.df = self.df.toDF(*colmap)
        return self
    
    def camelCase_to_snake_case(self):
        normalized_columns = []
        col_snake_case = ''
        for col in self.df.columns:
            col_snake_case = ''
            if not col.islower():
                for char in col:
                    if char.isupper():
                        col_snake_case += '_' + char.lower()
                        continue
                    col_snake_case += char
            normalized_columns.append(col_snake_case)
        Refine.renamecols(self, normalized_columns)
        return self
    
    def addcol(self, colname, datatype, colvalue=None):
        self.df = self.df.withColumn(colname, lit(colvalue).cast(datatype))
        return self
    
    def add_constcol(self, colname, datatype, colvalue=None):
        self.df = self.df.withColumn(colname, lit(colvalue).cast(datatype))
        return self
    
    def merge_cols(self, columns=[None], newcolname=None, dropcolumns=True, separator=' '):
        if(columns == None):
            raise Exception("Columns must be a list of non null values")
        if(newcolname is None):
            newcolname = '_'.join(columns)

        self.df = self.df.withColumn(newcolname, 
                                     reduce(lambda x, y: concat(x, lit(separator), y), [col(c) for c in columns]))
        if dropcolumns:
            self.df = self.df.drop(*columns)
        return self
    
    def load(self):
        return self.df